# <center> Tests de différentes architectures MobileNet

On compare ici différentes architectures avec exactement les mêmes hyperparamètres que pour ResNet18 c'est à dire : 
- Même split train / val / test
- Même preprocessing
- Même nombre d’epochs
- Même optimizer (Adam)
- Même early stopping

Stratégie en 2 phases :

Phase 1 — Head-only training (comme ResNet18, on freeze tous les paramètres du modèle et on entraine uniquement la couche finale)
→ Permet comparaison équitable

Phase 2 — Fine-tuning partiel (optionnel mais intéressant, on débloque les 1-2 dernières couches mais risque de surapprentissage)
→ Permet voir si les performances stagnent à cause du gel

Pour chaque modèle : 
| Modèle | Accuracy | F1 weighted | Params | Temps d’entraînement |
| ------ | -------- | ----------- | ------ | -------------------- |


Nous nous consacrons dans ce notebook à tester différents arcitecture MobileNet :  
- MobileNet  
- EfficientNet-B1   

Normalement EfficientNet équilibre parfaitement la profondeur, la largeur et la résolution de l'image. 

### 0. Préparation des hyper-paramètres et du dataset

#### Imports de base

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import sys
from pathlib import Path

# Pour que notre archi fonctionne avec google colab

!git clone https://github.com/julietteabalain-cloud/Reconnaissance-de-mouvement-artistique.git
!cd /content/Reconnaissance-de-mouvement-artistique && git pull
%cd /content/Reconnaissance-de-mouvement-artistique
import sys
sys.path.append(".")  # pour que src/ soit importable

PROJECT_ROOT = Path().resolve().parent
sys.path.append(str(PROJECT_ROOT))
DATA_ROOT = PROJECT_ROOT / "data"

In [ ]:
from src.dataset_dl import ArtDataset
from src.train import train_model, train_one_epoch, validate_one_epoch

from src.dataset import load_df_train_test_val, load_df
from src.preprocessing import clean_dataset

from src.models import get_model
from src.evaluate import *
from src.utils import set_seed

#Fixer l'initialisation aléatoire pour la reproductibilité
set_seed(42)

#pour avoir acces au GPU si dispo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from pathlib import Path

PROJECT_ROOT = Path("/content/deepl-projet")
DATA_ROOT = Path("/content/drive/MyDrive/DeepLearning/WikiArt_Subset")


df_test, df_train, df_val = load_df_train_test_val(DATA_ROOT)
df = load_df(DATA_ROOT)

df, df_train, df_val, df_test = clean_dataset(df, df_train, df_val, df_test)

#### Dataset de deep learning

In [ ]:
transform_train = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
    # ajout de data augmentation pour le training set
    # transforms.RandomHorizontalFlip(),
    # transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    # transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1)
])

transform_val = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ),
])


In [ ]:
IMAGE_ROOT_TRAIN = DATA_ROOT / "train"
IMAGE_ROOT_VAL = DATA_ROOT / "val"
IMAGE_ROOT_TEST = DATA_ROOT / "test"

train_dataset = ArtDataset(
    df_train,
    IMAGE_ROOT_TRAIN,
    transform=transform_train
)

val_dataset = ArtDataset(
    df_val,
    IMAGE_ROOT_VAL,
    transform=transform_val
)

test_dataset = ArtDataset(
    df_test,
    IMAGE_ROOT_TEST,
    transform=transform_val
)

In [ ]:
BATCH_SIZE = 32

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2,     # ajuster selon ton CPU
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,     # ajuster selon ton CPU
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=2,     # ajuster selon ton CPU
    pin_memory=True
)

In [ ]:
class EarlyStopping:
    def __init__(self, patience=3):
        self.patience = patience
        self.best_loss = float("inf")
        self.counter = 0
        self.stop = False

    def step(self, val_loss):
        if val_loss < self.best_loss:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.stop = True

### 1. MobileNet v2

→ Architecture légère
→ Depthwise separable convolutions
→ Très peu de paramètres

Le problème nécessite-t-il une grande capacité ou un modèle léger suffit-il ?

#### 2.1 Charger le modèle

In [ ]:
model_mn = get_model("mobilenet_v2", num_classes=23)
model_mn = model_mn.to(device)

num_classes = df_train["style_encoded"].nunique()

# ajout de label smoothing pour la cross entropy loss
# label smoothing permet de rendre le modèle moins confiant dans ses prédictions,
# ce qui peut aider à améliorer la généralisation et réduire le surapprentissage
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

optimizer = torch.optim.Adam(
    model_mn.fc.parameters(),
    lr=1e-3,
    weight_decay=1e-4
)

#### 2.2 Entrainement 

In [ ]:
# from src.utils import EarlyStopping

early_stopping = EarlyStopping(patience=3)

NUM_EPOCHS_FREEZE = 10
history_freeze_02 = train_model(
    model_mn,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    device,
    num_epochs=NUM_EPOCHS_FREEZE,
    early_stopping=early_stopping
)


#### 2.3 Evaluation

In [ ]:
train_acc = history_freeze_mn["train_acc"]
val_acc   = history_freeze_mn["val_acc"]

plt.plot(train_acc)
plt.plot(val_acc)
plt.legend(["Train", "Validation"])
plt.title("MobileNetv2 Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.show()

In [ ]:
#Confusion matrix :

class_names = sorted(df_train["style"].unique())

cm = compute_confusion_matrix(
    model_mn,
    val_loader,
    device,
    class_names
)

plot_confusion_matrix(cm, class_names)


In [ ]:
class_names = sorted(df_train["style"].unique())

acc_per_style = accuracy_per_class(
    model_mn,
    val_loader,
    device,
    class_names
)

results = list(zip(class_names, acc_per_style))
results = sorted(results, key=lambda x: x[1], reverse=True)

for style, acc in results:
    print(f"{style}: {acc:.3f}")


In [ ]:
visualize_accuracy_per_style(results)

#### 1.6.2 Evaluation sur l'ensemble de test

In [ ]:
best_model_weights = model_mn.state_dict()

test_acc, test_cm, report = evaluate_model(model_mn, test_loader, device)

print(f"Test Accuracy: {test_acc:.3f}")
print("Classification Report:")
print(report)

In [ ]:
print("Test Confusion Matrix:")
plot_confusion_matrix(test_cm, class_names)